# Love Behavior Implementation

**Step 1: Create display widgets**

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

# Create widgets for displaying images
display_color = widgets.Image(format='jpeg', width='30%')
display_depth = widgets.Image(format='jpeg', width='30%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

# Convert numpy array to jpeg coded data for displaying
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

**Step 2: Initialize camera and implement love behavior**

The robot will exhibit love behavior by:
- Approaching objects when they are far away (> 1000mm)
- Maintaining an optimal distance (700-1000mm)
- Backing away slightly if too close (< 700mm)

This creates an attraction effect where the robot seeks to stay near objects at a comfortable distance.

In [ ]:
import numpy as np
import pyzed.sl as sl
import threading
import motors

# Initialize the Robot class
robot = motors.MotorsYukon(mecanum=False)

class Camera():
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        # Create a InitParameters object and set configuration parameters
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER

        # Open the camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

        # Create and set RuntimeParameters after opening the camera
        self.runtime = sl.RuntimeParameters()

        # Flag to control the thread
        self.thread_runnning_flag = False

        # Get the height and width
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)

    def _capture_frames(self):
        while(self.thread_runnning_flag == True):
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                # Retrieve Left image
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                # Retrieve depth map. Depth is aligned on the left image
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value = self.image.get_data()
                # Scale for real-time data displaying
                scale = 0.1
                resized_image = cv2.resize(self.color_value, None, fx=scale, fy=scale, 
                                          interpolation=cv2.INTER_AREA)
                cv2.circle(resized_image, (int(self.width*scale//2), int(self.height*scale//2)), 
                          1, (0, 255, 0))
                display_color.value = bgr8_to_jpeg(resized_image)
                
                self.depth_image = np.asanyarray(self.depth.get_data())

                # Process depth image for love behavior
                depth_image_test = self.depth_image.copy()
                depth_image_test = np.nan_to_num(depth_image_test, nan=0.0).astype(np.float32)
                
                # Define central area (similar to Tutorial 3 Part B)
                depth_image_test[:94, :] = 0
                depth_image_test[282:, :] = 0
                depth_image_test[:, :168] = 0
                depth_image_test[:, 504:] = 0

                # Filter depth values
                depth_image_test[depth_image_test < 100] = 0
                depth_image_test[depth_image_test > 2000] = 0
                
                depth_colormap = cv2.applyColorMap(
                    cv2.convertScaleAbs(depth_image_test, alpha=0.03), 
                    cv2.COLORMAP_JET)
                
                # LOVE BEHAVIOR - approach and maintain optimal distance
                if(depth_image_test.max() == 0):
                    robot.stop()
                    cv2.putText(depth_colormap, 'No target detected', (250, 188), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2, cv2.LINE_AA)
                    print('No target - stopped', end='\r')
                else:
                    distance = depth_image_test[depth_image_test != 0].min()
                    
                    # Love behavior: maintain optimal distance around 700-1000mm
                    if distance > 1500:
                        robot.forward(0.5)
                        cv2.putText(depth_colormap, f'Approaching {distance:.0f}mm', (220, 188),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2, cv2.LINE_AA)
                        print(f'Love: Fast approach {distance:.0f}mm', end='\r')
                    elif distance > 1000:
                        robot.forward(0.3)
                        cv2.putText(depth_colormap, f'Moving closer {distance:.0f}mm', (220, 188),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 180, 0), 2, cv2.LINE_AA)
                        print(f'Love: Slow approach {distance:.0f}mm', end='\r')
                    elif distance > 700:
                        robot.stop()
                        cv2.putText(depth_colormap, f'Optimal {distance:.0f}mm', (250, 188),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)
                        print(f'Love: Optimal distance {distance:.0f}mm', end='\r')
                    else:
                        robot.backward(0.25)
                        cv2.putText(depth_colormap, f'Too close {distance:.0f}mm', (240, 188),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2, cv2.LINE_AA)
                        print(f'Love: Backing up {distance:.0f}mm', end='\r')
                    
                resized_depth_colormap = cv2.resize(depth_colormap, None, fx=scale, fy=scale,
                                                    interpolation=cv2.INTER_AREA)
                display_depth.value = bgr8_to_jpeg(resized_depth_colormap)
                
    def start(self):
        if self.thread_runnning_flag == False:
            self.thread_runnning_flag = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False
            self.thread.join()
            robot.stop()

camera = Camera()
camera.start()

**To stop the camera and robot:**

In [ ]:
camera.stop()

1. **No target detected**: Robot stops and waits
2. **Target far away (> 1500mm)**: Robot approaches at higher speed (0.5)
3. **Target at medium distance (1000-1500mm)**: Robot approaches slowly (0.3)
4. **Target at optimal distance (700-1000mm)**: Robot stops, maintaining comfortable proximity
5. **Target too close (< 700mm)**: Robot backs away gently (0.25)